In [1]:
import heapq
import math
import urllib.request
from itertools import groupby


def Input(day):
    "Open input file for the day"
    file = '2017/input{0}.txt'.format(day)
    try:
        return open(file)
    except FileNotFoundError:
        return urllib.request.urlopen(
            'http:/adventofcode.com/2017/day/{0}/input'.format(day))
        

# Day 1: Inverse Captcha
The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

In [2]:
def parse(text):
    #append the first digit on the end, and group like digits
    return groupby(text + text[0])

def group(sequence):
    #return list of (digit, # of repetitions) tuples
    return [(int(digit),len(list(group))-1) for digit, group in parse(sequence)]

def inverse_captcha(groups):
    #sum all repeated digits
    return sum([digit*reps for digit,reps in groups if reps >= 1])

assert(inverse_captcha(group('1122')) == 3)
assert(inverse_captcha(group('1111')) == 4)
assert(inverse_captcha(group('1234')) == 0)
assert(inverse_captcha(group('91212129')) == 9)

inverse_captcha(group(Input(1).read().rstrip()))

1097

**Part 2**: Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. That is, if your list contains 10 items, only include a digit in your sum if the digit 10/2 = 5 steps forward matches it. Fortunately, your list has an even number of elements.

In [3]:
def parse(text):
    #pair digits and their shifted partners
    size = len(text)
    shift = get_shifter(size)
    return [(int(text[i]),int(text[shift(i)])) for i in range(size//2)]

def get_shifter(length):
    #given an index return the halfway shifted index
    return lambda x : (x + length // 2) % length

def halfway_captcha(values):
    return sum([i*2 for i,j in values if i == j])

assert(halfway_captcha(parse('1212')) == 6)
assert(halfway_captcha(parse('1221')) == 0)
assert(halfway_captcha(parse('123425')) == 4)
assert(halfway_captcha(parse('123123')) == 12)
assert(halfway_captcha(parse('12131415')) == 4)

halfway_captcha(parse(Input(1).read().rstrip()))


1188

# Day 2: Corruption Checksum
The spreadsheet consists of rows of apparently-random numbers. To make sure the recovery process is on the right track, they need you to calculate the spreadsheet's checksum. For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

In [6]:
def parse(text):
    return [[int(x) for x in line.split()] for line in text.splitlines()]

def range_diff(values):
    values.sort()
    return values[-1] - values[0]
    
def checksum(rows):
    return sum([range_diff(row) for row in rows])

assert(parse('''1    2    3
                4    5    6''') == [[1,2,3],[4,5,6]])
assert(range_diff([1,2,3,4]) == 3)
assert(checksum([[5,1,9,5],[7,5,3],[2,4,6,8]]) == 18)

checksum(parse(Input(2).read().rstrip()))

41887

**Part 2:** It sounds like the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.

In [17]:
def even_quotient(values):
    values.sort()
    for i in range(len(values)):
        for j in range(i+1,len(values)):
            if values[j] % values[i] == 0:
                return values[j] // values[i] 
        
def checksum(rows):
    return sum([even_quotient(row) for row in rows])

assert(even_quotient([2,3,4,5]) == 2)
assert(checksum([[5,9,2,8],[9,4,7,3],[3,8,6,5]]) == 9)

checksum(parse(Input(2).read().rstrip()))

226

# Day 3: Spiral Memory

In [95]:
def distance(n):
    next_odd_sqrt = 2 * (math.ceil(math.sqrt(n)) // 2 + 1) - 1
    max_layer_value = next_odd_sqrt**2
    side_len = next_odd_sqrt - 1 #also the number of steps for a corner to reach 1
    corners = [ max_layer_value - i*side_len for i in range(4)]
    distances = [abs(corner - n) for corner in corners]
    return side_len - min(distances)

assert(distance(1) == 0)
assert(distance(12) == 3)
assert(distance(23) == 2)
assert(distance(1024) == 31)

distance(361527) 

326

**Part 2:**

# Day 4: High-Entropy Passphrases
A new system policy has been put in place that requires all accounts to use a passphrase instead of simply a password. A passphrase consists of a series of words (lowercase letters) separated by spaces.

To ensure security, a valid passphrase must contain no duplicate words.

In [96]:
def validate(passphrase):
    passphrase = passphrase.split()
    return len(set(passphrase)) == len(passphrase)
    

assert(validate('aa bb cc dd ee') == True)
assert(validate('aa bb cc dd aa') == False)

sum([validate(phrase) for phrase in Input(4).read().rstrip().splitlines()])

466

**Part 2:** For added security, yet another system policy has been put in place. Now, a valid passphrase must contain no two words that are anagrams of each other - that is, a passphrase is invalid if any word's letters can be rearranged to form any other word in the passphrase.

In [110]:
def validate(passphrase):
    #sort each word alphabetically
    passphrase = list(map(lambda x : ''.join(sorted(x)), passphrase.split()))
    return len(set(passphrase)) == len(passphrase)

assert(validate('abcde fghij') == True)
assert(validate('abcde xyz ecdab') == False)
assert(validate('a ab abc abd abf abj') == True)
assert(validate('iiii oiii ooii oooi oooo') == True)
assert(validate('oiii ioii iioi iiio') == False)

sum([validate(phrase) for phrase in Input(4).read().rstrip().splitlines()])

251

# Day 5: A Maze of Twisty Trampolines, All Alike

In [11]:
def parse(text):
    return [int(line) for line in text.splitlines()]

def execute(instructions):
    pointer, counter = 0, 0
    while pointer < len(instructions):
        counter += 1
        temp = pointer 
        pointer += instructions[pointer]
        instructions[temp] += 1
    return counter

assert(execute([0,3,0,1,-3]) == 5)
execute(parse(Input(5).read().rstrip()))

358309

**Part Two:**

In [10]:
def execute(instructions):
    pointer, counter = 0, 0
    while pointer < len(instructions):
        counter += 1
        temp = pointer 
        pointer += instructions[pointer]
        if instructions[temp] >= 3:
            instructions[temp] -= 1
        else:
            instructions[temp] += 1
    return counter

assert(execute([0,3,0,1,-3]) == 10)
execute(parse(Input(5).read().rstrip()))

28178177